# TO DO

1. finish objective function - look at code on how to call dataloaders and our custom classes on chatgpt
2. get it to plot train loss vs val loss on best model???
3. get it to spit out accuracy, hit rate, f1 and ROC AUC. get it to plot ROC AUC graph??
4. get it to output best HP combo
5. check paper for other tips and tricks they used.
6. look into Cell-based Architectures: Explore more structured approaches for dynamic architectures, such as cell-based architectures (like those used in NASNet or EfficientNet), which offer a balance between flexibility and control



---

# Intro

**Plan**: Import credit card fraud data. Use encoder only transformer network for classifying time series credit card data

**Purpose/Intro**: Task is to develop transformer architecture proof of concept for potential application at work, detecting fraud. In a normal data science project it might be considered best practice to begin with more interpretable models first, for research purposes, but this project is solely for the purpose of assessing the viability of a transformer for this task.

**Hypothesis**: The attention mechanism of the transformer, when combined with an appropriate positional embedding method, is able to capture both long-term and short-term dependencies in time series credit-card fraud data.

**Methodology**: Using cross valdiation techniques on test dataset to calculate appropriate accuracy metrics (adjusting for the significant class imbalance for the dataset), with an aim to assess the viability of transformer networks for fraud classification.

Credit to the below paper, **referred to as Source 1**, for the methodology design: Yu, C., Xu, Y., Cao, J., Zhang, Y., Jin, Y. and Zhu, M. (2024) 'Credit Card Fraud Detection Using Advanced Transformer Model', arXiv preprint arXiv:2406.03733. Available at: https://arxiv.org/abs/2406.03733 (Accessed: 18 December 2024)

This paper has demonstrated the utility for the transformer that we are about to create, by comparing the methodology with various other shallow learning techniques. In future projects I aim to validate this myself.





---

# Data Sourcing and Package loading



In [ ]:

#import packages:

import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

from google.colab import drive

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  # Check if drive is mounted by looking for the mount point in the file system.
  import os
  if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

#basics
import os
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip install optuna
import optuna

#cuML
from cuml.preprocessing import MinMaxScaler
import cudf
import cupy as cp
from imblearn.over_sampling import SMOTE  # GPU-enabled SMOTE with cuML


#table one
!pip install tableone
from tableone import TableOne

#torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset


#sklearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split, RepeatedStratifiedKFold, KFold
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler

from imblearn.over_sampling import RandomOverSampler

In [ ]:
data_set_filepath = '/content/drive/MyDrive/Colab_Notebooks/Data/creditcard.feather'

df = pd.read_feather(data_set_filepath)

missing_values = df.isnull().sum()

if missing_values.any():  # Check if any missing values exist
    print("Missing Values Found:")
    print(missing_values)
else:
    print("No Missing Values Found")

columns = df.columns.tolist()

print(f"The dataset lenghth is {str(len(df))}")
print(f"The number of columns is {str(len(columns))}")
print(f"The column names are {str(columns)}")
df.head(10)

#table1 = TableOne(df, columns=columns, groupby= 'Class', pval=True)
#print(table1)

data = df







---


# Data loading and preprocessing:
In Source 1 (listed above), it was found that there are performance boosts associated with removing outliers, as it may help with overfitting. This will be done on the training data only. This is to prevent information leakage from our training set.

The source also suggests there is value in oversampling the minority class. This may be due to the unique challenges of such a large class imbalance. This will be done on the training data only. This is to prevent information leakage from our training set.

In addition, we will min-max scale our validation and training sets, and apply this same scaling to the test data.

In [13]:
# **Set device for GPU acceleration**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **Error warning if no GPU is detected**
if device.type != 'cuda':
    print("WARNING: GPU is not available. The model will run on the CPU, which might be slower.")
else:
    print("Cuda setup successful")

#separates all data into data and targets
X = data.iloc[:, :-1]  # Features (all columns except the last one)
y = data.iloc[:, -1]   # Labels (the last column)

#sets X and y as the training + validation set, and x_test and y_test as the test set
X, X_test, y, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y  # stratify for class imbalance
)

# Data Preprocessing Transformation Class
class DataPreprocessingTransform:
    def __init__(self):
        self.scaler = MinMaxScaler()

    def fit_transform(self, X_train):
        """
        Fit the scaler on the training set and transform it.
        """
        X_train_cudf = cudf.DataFrame(X_train)
        X_scaled = self.scaler.fit_transform(X_train_cudf)
        return X_scaled  # Returns a cuDF DataFrame

    def transform(self, X):
        """
        Apply the scaling transformation based on the training set scaling.
        """
        X_cudf = cudf.DataFrame(X)
        X_scaled = self.scaler.transform(X_cudf)
        return X_scaled  # Returns a cuDF DataFrame



# Custom PyTorch Dataset
class CustomDataset(Dataset):
    def __init__(self, X, y, transform=None):
        """
        Custom dataset to handle GPU-enabled data directly.
        """
        self.X = X  # Should be cuDF or GPU tensor
        self.y = y  # Should be cuDF or GPU tensor
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        sample_X = self.X.iloc[index]
        sample_y = self.y.iloc[index]

        if self.transform:
            sample_X = self.transform(sample_X)

        # Convert to GPU tensor if not already
        sample_X = cp.array(sample_X.values) if not isinstance(sample_X, cp.ndarray) else sample_X
        sample_y = cp.array(sample_y.values) if not isinstance(sample_y, cp.ndarray) else sample_y

        return torch.tensor(sample_X, dtype=torch.float32, device="cuda"), torch.tensor(sample_y, dtype=torch.float32, device="cuda")

    def _apply_smote(self, X, y):
        """
        Applies GPU-accelerated SMOTE and returns cuDF DataFrame and Series.
        """
        X_cudf = cudf.DataFrame(X)
        y_cudf = cudf.Series(y)

        smote = SMOTE(sampling_strategy="auto", random_state=42)
        X_resampled, y_resampled = smote.fit_resample(X_cudf, y_cudf)

        return X_resampled, y_resampled  # Returns cuDF DataFrame and Series


# Function to remove outliers using IQR
def remove_outliers_gpu(X):
    """
    Removes outliers from the dataset based on the IQR method using cuDF.
    This function is optimized for GPU processing.

    Parameters:
    - X: cuDF DataFrame

    Returns:
    - X_filtered: cuDF DataFrame with outliers removed
    """
    # Compute Q1, Q3, and IQR
    Q1 = X.quantile(0.25)
    Q3 = X.quantile(0.75)
    IQR = Q3 - Q1

    # Calculate bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter out rows outside bounds
    X_filtered = X.fillna(X.median())  # Fill NaNs first
    mask = (X >= lower_bound) & (X <= upper_bound)  # Boolean mask for filtering
    X_filtered = X.where(mask, other=None).dropna()  # Apply mask and drop rows with NaNs

    return X_filtered



Cuda setup successful




---
# Transformer Model



We are going to implement the transformer and optimise the hyperparameters using the Optuna package. As per the paper listed above, we will resample from the minority class when training.

In [14]:
# **Transformer Model with Batch Normalization and Residual Connections**
class FraudDetectionTransformer(nn.Module):

      #init method dynamically builds model architecture
      def __init__(self, input_dim, embed_dim, num_heads, ff_dim_base, dropout, ff_dropout, activation_function, num_ff_layers, use_batchnorm, use_layernorm):
          super(FraudDetectionTransformer, self).__init__()

          self.embedding = nn.Linear(input_dim, embed_dim)  # Embedding layer
          self.use_batchnorm = use_batchnorm # Store whether to use batchnorm

          self.batch_norm = nn.BatchNorm1d(input_dim) if self.use_batchnorm else None # Batch Normalization before embedding

          # Store the activation function
          self.activation_function = activation_function

          # Multi-head Attention Layer
          self.multihead_attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout)

          # Dynamically create feedforward network in __init__
          ff_network = []
          for i in range(num_ff_layers):
              ff_dim = ff_dim_base * (2 ** i)
              ff_network.extend([
                  nn.Linear(embed_dim if i == 0 else ff_dim // 2, ff_dim),
                  self.get_activation_function(),
                  nn.Dropout(ff_dropout)
              ])

          self.ff_network = nn.Sequential(*ff_network)  # Store the created network

          # Layer Normalization after the dynamic feedforward network
          self.layer_norm = nn.LayerNorm(embed_dim) if use_layernorm else None

          self.pooling = nn.AdaptiveAvgPool1d(1)  # Global average pooling
          self.fc = nn.Linear(embed_dim, 1)  # Fully connected layer for classification
          self.sigmoid = nn.Sigmoid()  # Sigmoid activation for binary classification


      def forward(self, x):
          # Apply batch normalization if specified
          if self.use_batchnorm:
              x = self.batch_norm(x)

          # Apply embedding layer
          embedded_x = self.embedding(x)

          # Add sequence dimension for the transformer
          embedded_x = embedded_x.unsqueeze(1)

          # Transpose for multi-head attention (sequence_length, batch_size, embed_dim)
          embedded_x = embedded_x.permute(1, 0, 2)

          # Pass through multi-head attention and add residual connection
          attn_output, _ = self.multihead_attn(embedded_x, embedded_x, embedded_x)
          attn_output = attn_output + embedded_x

          # Pass through feedforward layers with residual connection
          transformer_output = self.ff_network(attn_output.permute(1, 0, 2))  # (batch_size, sequence_length, embed_dim)
          transformer_output = transformer_output + attn_output.permute(1, 0, 2)

          # Layer normalization if specified
          if self.layer_norm:
              transformer_output = self.layer_norm(transformer_output)

          # Global pooling over the sequence length dimension
          pooled_output = self.pooling(transformer_output.permute(0, 2, 1)).squeeze()

          # Final fully connected layer and sigmoid activation
          fc_output = self.fc(pooled_output)
          return self.sigmoid(fc_output)


# **Training Function with Early Stopping**
def train_model_with_early_stopping(model, train_loader, val_loader, epochs, lr, patience):

    model = model.to(device) #switch to GPU
    optimizer = torch.optim.Adam(model.parameters(), lr=lr) #select Adam for optimization
    criterion = nn.BCELoss() #set loss funct

    #history
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    patience_counter = 0
    best_model = None


    for epoch in range(epochs):
        model.train() #set model to training mode
        train_loss = 0.0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)  # Move inputs to GPU
            labels = labels.to(device)  # Move labels to GPU
            optimizer.zero_grad()    #zero gradients
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_losses.append(train_loss / len(train_loader))

        # Validation phase
        model.eval()
        val_loss = 0.0

        with torch.no_grad(): #no gradient needed for val phase
            for inputs, labels in val_loader: #unpack data
                inputs = inputs.to(device)  # Move inputs to GPU
                labels = labels.to(device)  # Move labels to GPU
                outputs = model(inputs).squeeze() #squeeze removes unecessary dimensions of tensors e.g. [batch_size,1] -> [batch_size]
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        val_losses.append(val_loss / len(val_loader)) #avg loss

        # Early stopping check
        if val_losses[-1] < best_val_loss:
            best_val_loss = val_losses[-1]
            best_model = model.state_dict()  # Save the best model
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                break

        print(f"Epoch {epoch + 1}/{epochs} - Train Loss: {train_losses[-1]:.4f} - Val Loss: {val_losses[-1]:.4f}")

    model.load_state_dict(best_model)  # Load the best model before returning
    return train_losses, val_losses, best_model

# **Validation Metrics Calculation**
def evaluate(model, data_loader):
    model.eval() #set to eval mode
    y_true = []
    y_pred = []

     with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs).squeeze()

            # Keep data as tensors on the GPU
            y_true.extend(labels)
            y_pred.extend(outputs)

    # Convert predictions to binary format for accuracy and hit rate
    y_pred_binary = [1 if pred >= 0.5 else 0 for pred in y_pred]

    # Metrics
    accuracy = accuracy_score(y_true, y_pred_binary)
    hit_rate = np.sum(np.logical_and(np.array(y_true) == 1, np.array(y_pred_binary) == 1)) / np.sum(np.array(y_true) == 1)
    roc_auc = roc_auc_score(y_true, y_pred)

    return accuracy, hit_rate, roc_auc

# **Plotting Function**
def plot_losses(train_losses, val_losses):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label="Training Loss")
    plt.plot(val_losses, label="Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()
    plt.show()


# Modify the objective function to save the best model
def objective(trial):
    # Define hyperparameters to tune
    num_heads = trial.suggest_int("num_heads", low=4, high=8, step=2)
    embed_dim = trial.suggest_int("embed_dim", low=64, high=512, step=num_heads)
    ff_dim_base = trial.suggest_int("ff_dim_base", low=64, high=512, step=64)
    dropout = trial.suggest_float("dropout", low=0.1, high=0.5, step=0.1)
    ff_dropout = trial.suggest_float("ff_dropout", low=0.1, high=0.5, step=0.1)
    activation_function = trial.suggest_categorical("activation_function", ["relu", "tanh", "sigmoid"])
    num_ff_layers = trial.suggest_int("num_ff_layers", low=1, high=3, step=1)
    use_batchnorm = trial.suggest_categorical("use_batchnorm", [True, False])
    use_layernorm = trial.suggest_categorical("use_layernorm", [True, False])
    batch_size = trial.suggest_int("batch_size", 64, 512, step=64)

    # Print the hyperparameters being trialed
    print(f"Trial {trial.number}:")
    print(f"  num_heads: {num_heads}")
    print(f"  embed_dim: {embed_dim}")
    print(f"  ff_dim_base: {ff_dim_base}")
    print(f"  dropout: {dropout}")
    print(f"  ff_dropout: {ff_dropout}")
    print(f"  activation_function: {activation_function}")
    print(f"  num_ff_layers: {num_ff_layers}")
    print(f"  use_batchnorm: {use_batchnorm}")
    print(f"  use_layernorm: {use_layernorm}")
    print(f"  batch_size: {batch_size}")
    print("-" * 50)

    # K-Fold Cross Validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    fold_scores = []
    best_val_score = -float('inf')
    best_model_state_dict = None

    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        # Preprocess the data
        transform = DataPreprocessingTransform()
        X_train_scaled = transform.fit_transform(X_train)
        X_val_scaled = transform.transform(X_val)

        # Remove outliers
        X_train_filtered = X_train_scaled.apply(remove_outliers, axis=1) # Apply outlier removal to each row
        X_val_filtered = X_val_scaled.apply(remove_outliers, axis=1)

        # Reset index for BOTH X_train_filtered and y_train after outlier removal
        X_train_filtered = X_train_filtered.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)

        # Now align y_train based on the shared index after outlier removal
        # This will ensure that both DataFrames have the same number of rows and consistent indexing
        common_index = X_train_filtered.index.intersection(y_train.index)
        X_train_filtered = X_train_filtered.loc[common_index]
        y_train = y_train.loc[common_index]

        # Create PyTorch datasets
        train_dataset = CustomDataset(X_train_filtered, y_train, resample=True)
        val_dataset = CustomDataset(X_val_filtered, y_val)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        # Initialize the model, loss function, and optimizer
        model = FraudDetectionTransformer(
            input_dim=input_dim,
            embed_dim=embed_dim,
            num_heads=num_heads,
            ff_dim_base=ff_dim_base,
            dropout=dropout,
            ff_dropout=ff_dropout,
            activation_function=activation_function,
            num_ff_layers=num_ff_layers,
            use_batchnorm=use_batchnorm,
            use_layernorm=use_layernorm
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        criterion = nn.BCELoss()

        # Train the model
        train_model(model, train_loader, val_loader, optimizer, criterion)
        val_score = evaluate_model(model, val_loader)

        # Keep track of the best model
        if val_score > best_val_score:
            best_val_score = val_score
            best_model_state_dict = model.state_dict()

        fold_scores.append(val_score)

    avg_fold_score = sum(fold_scores) / len(fold_scores)

    # Save the best model after all folds
    if best_model_state_dict is not None:
        torch.save(best_model_state_dict, 'best_model.pth')

    return avg_fold_score

# Run Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

# Output the best parameters
print("Best hyperparameters:", study.best_params)



#=============================================== Test on Test Set ===============================================#

# Evaluate on holdout test set
# Evaluate on holdout test set
best_params = study.best_params

best_model = FraudDetectionTransformer(
            input_dim=best_params['input_dim'],
            embed_dim=best_params['embed_dim'],
            num_heads=best_params['num_heads'],
            ff_dim_base=best_params['ff_dim_base'],
            dropout=best_params['dropout'],
            ff_dropout=best_params['ff_dropout'],
            activation_function=best_params['activation_function'],
            num_ff_layers=best_params['num_ff_layers'],
            use_batchnorm=best_params['use_batchnorm'],
            use_layernorm=best_params['use_layernorm']
        ).to(device)

# scale the data based on the training-set-fit but don't resample
# 1. Data Preprocessing for Test Set (using same transformations as in objective)
transform = DataPreprocessingTransform()
X_scaled = transform.fit_transform(X)  # Fit on training data
X_test_scaled = transform.transform(X_test)  # Transform test data

# Remove outliers (if applied in objective)
X_filtered = X_scaled.apply(remove_outliers, axis=1).reset_index(drop=True)  #reset index for outlier removal
y = y.reset_index(drop=True)

# Now align y_train based on the shared index after outlier removal
# This will ensure that both DataFrames have the same number of rows and consistent indexing
common_index = X_filtered.index.intersection(y.index)
X_filtered = X_filtered.loc[common_index]
y = y.loc[common_index]

X_test_filtered = X_test_scaled.apply(remove_outliers, axis=1)  #reset index for outlier removal

# 2. Create Datasets and DataLoaders
train_val_dataset = CustomDataset(X_filtered, y, transform=None, resample=True)  # Resample for training
test_dataset = CustomDataset(X_test_filtered, y_test, transform=None, resample=False)

train_val_loader = DataLoader(train_val_dataset, batch_size=best_params["batch_size"], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=best_params["batch_size"], shuffle=False)

# 3. Initialize Final Model and Optimizer
final_model = FraudDetectionTransformer(
    input_dim=X.shape[1],  # Assuming input_dim wasn't tuned
    # ... other hyperparameters from best_params ...
).to(device)

optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params.get("lr", 1e-3))  # Use best lr or default
criterion = nn.BCELoss()

# 4. Train Final Model (using train_val_loader and test_loader for validation/early stopping)
train_losses, val_losses, best_model_state_dict = train_model_with_early_stopping(
    final_model, train_val_loader, test_loader, epochs=epochs, lr=best_params.get("lr", 1e-3), patience=patience
)

# Load best model weights
final_model.load_state_dict(best_model_state_dict)

# 5. Evaluate on Test Set
test_accuracy, test_hit_rate, test_roc_auc = evaluate(final_model, test_loader)

print("Test Accuracy:", test_accuracy)
print("Test Hit Rate:", test_hit_rate)
print("Test ROC AUC:", test_roc_auc)



[I 2024-12-19 16:08:49,955] A new study created in memory with name: no-name-eac7626b-9e09-4f78-919d-ea6dbda603ab


Trial 0:
  num_heads: 4
  embed_dim: 100
  ff_dim_base: 256
  dropout: 0.1
  ff_dropout: 0.5
  activation_function: relu
  num_ff_layers: 2
  use_batchnorm: True
  use_layernorm: False
  batch_size: 448
--------------------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
[W 2024-12-19 16:16:54,181] Trial 0 failed with parameters: {'num_heads': 4, 'embed_dim': 100, 'ff_dim_base': 256, 'dropout': 0.1, 'ff_dropout': 0.5, 'activation_function': 'relu', 'nu

ValueError: Input X contains NaN.
SMOTE does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values



---

# Auto commit to github

In [ ]:
import datetime
import os

# Navigate to the repository directory (if not already there)
%cd /content/drive/MyDrive/Colab_Notebooks/Deep_Learning_Practice

with open('/content/drive/MyDrive/IAM/PAT.txt', 'r') as file:
      github_pat = file.read().strip()
os.environ['GITHUB_PAT'] = github_pat

!git remote add origin "https://github.com/archiegoodman2/machine_learning_practice"

# Replace with your actual username and email (or configure globally)
USERNAME="archiegoodman2"
EMAIL="archiegoodman2011@gmail.com"

# Set global username and email configuration
!git config --global user.name "$USERNAME"
!git config --global user.email "$EMAIL"

now = datetime.datetime.now()
current_datetime = now.strftime("%Y-%m-%d %H:%M")

# Set remote URL using the PAT from environment variable
!git remote set-url origin https://{os.environ['GITHUB_PAT']}@github.com/archiegoodman2/machine_learning_practice.git

# Replace with your desired commit message
COMMIT_MESSAGE = str(current_datetime) + " " + " second attempt "

# Stage all changes
!git add .

# Commit the changes
!git commit -m "$COMMIT_MESSAGE"

# Push to origin
!git push origin master


/content/drive/MyDrive/Colab_Notebooks/Deep_Learning_Practice
error: remote origin already exists.
